In [1]:
import tensorflow as tf
import numpy as np

# tf.Variable()
```python
tf.Variable(
    initial_value=None,
    trainable=None,
    validate_shape=True,
    caching_device=None,
    name=None,
    variable_def=None,
    dtype=None,
    import_scope=None,
    constraint=None,
    synchronization=VariableSynchronization.AUTO,
    aggregation=VariableAggregation.NONE,
    shape=None)
```

**Docstring**

用于声明一个由程序操作的、保持着共享性和持久性的`Variable`对象。此外在`tf.device`作用域下可以通过调用`tf.Variable.read_value()`来手动缓存变量的值。更多细节详见[variable guide](https://tensorflow.org/guide/variable)

**Args:**

- initial_value: 张量或可转换为张量的 Python 对象，是`Variable`的初始化取值；`initial_value`也可以是也可以是不带参数的可调用函数，其在调用时返回用于初始化的对象，这种情况下`dtype`必须被指明；需要注意的是，若调用`init_ops.py`模块中的初始化函数，则该函数在调用前必须首先绑定到一个 shape 上

- trainable: 当 `synchronization`被设定为`ON_READ`时为`False`，否则默认为`True`；当其取值为`True`时，`GradientTapes`将自动记录这个变量的使用情况。

- validate_shape: 取值为`False`时，该变量可以用一个形状未知的值初始化，否则变量形状必须被指明

- caching_device: 字符串型变量，已被弃用；用于指明应在何处缓存变量以进行读取，默认为`Variable`所在的设备；取值不为`None`时则缓存到另一个设备上；通常缓存在`Ops`使用的变量驻留的设备上，以能够通过`Switch`和其他条件语句来减少重复的拷贝

- name: 变量的名称，默认为`'Variable'`并自动使该名称唯一化

- variable_def: `VariableDef`协议缓冲区；若取值不为`None`，则用于根据该变量在计算图中的节点，重新创建变量对象及其内容，但不会对计算图进行更改。`variable_def`和其他参数是互斥的，当其与`inital_value`同时被指明时会报错

- dtype: 当没有指明类型时，即其取值为`None`时，若`initial_value`为一张量，则该变量的数据类型将被保留；若`initial_value`不为张量，则数据类型由`convert_to_tensor`决定

- import_scope: 加给`Variable`的名称作用域，仅在从协议缓冲区初始化时使用

- constraint: 被`Optimizer`更新后作用在变量上的投影函数，如用于对该层权重的范数约束或值约束的函数；该函数必须以表示变量值的未投影张量作为输入，并返回投影值的张量，返回张量应与原张量形状相同。在进行异步分布式训练时，使用约束并不安全

- synchronization: 用于指明分布式变量是什么时候被聚集的，支持由`tf.VariableSynchronization`类定义的变量；默认情况下，同步状态设置为`AUTO`，并由当前的`DistributionStrategy`决定何时同步

- aggregation: 用于指明分布式变量是如何被聚集的，支持由`tf.VariableAggregation`类定义的变量

- shape: `None`时使用`initial_value`的形状，当设定为`tf.TensorShape(None)`时，即表示一个未指定的形状，该变量可被设定为任意形状


**File**:      \tensorflow\python\ops\variables.py

**Type**: VariableMetaclass

**Subclasses**: VariableV1, DistributedVariable, AggregatingVariable, AutoCastVariable

### Examples

In [4]:
x = tf.Variable(1.)  # => <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
print(x)
x = x + 1  # 该操作使 x 变为 tf.Tensor 类型，进而无法使用assign修改及赋值
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=2.0>
<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=2.5>
tf.Tensor(3.5, shape=(), dtype=float32)


In [8]:
x = tf.Variable(1., shape=tf.TensorShape(None))
print(x)
print(x.assign([[1.]]))

<tf.Variable 'Variable:0' shape=<unknown> dtype=float32, numpy=1.0>
<tf.Variable 'UnreadVariable' shape=<unknown> dtype=float32, numpy=array([[1.]], dtype=float32)>


In [ ]:
w = tf.Variable([[1.], [2.]])
x = tf.constant([[3., 4.]])
y = tf.matmul(w, x)
z = tf.sigmoid(w + x)
print(w, x, y, z, sep="\n\n")

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    trainable = tf.Variable(1.)
    non_trainable = tf.Variable(2., trainable=False)
    x1 = trainable * 2.
    x2 = non_trainable * 3.
print(tape.gradient(x1, trainable))
print(tape.gradient(x2, non_trainable))

In [ ]:
m = tf.Module()
m.v = tf.Variable([1.])
print(m.trainable_variables)
print(m.__dict__)

In [ ]:
v = tf.Variable(0.)
read_and_decrement = tf.function(lambda: v.assign_sub(0.1))
print(read_and_decrement())
print(read_and_decrement())

In [ ]:
class M(tf.Module):
    @tf.function
    def __call__(self, x):
        if not hasattr(self, "v"):  # Or set self.v to None in __init__
        self.v = tf.Variable(x)
        return self.v * x
m = M()
m(2.)
m(3.)
m.v
# See the `tf.function` documentation for details.

#  

<div align=center>

# 张量生成
</div>

# tf.constant

`tf.constant(value, dtype=None, shape=None, name='Const')`

__Docstring__

Creates a constant tensor from a tensor-like object.

Note: All eager `tf.Tensor` values are immutable (in contrast to
`tf.Variable`). There is nothing especially _constant_ about the value
returned from `tf.constant`. This function it is not fundamentally different
from `tf.convert_to_tensor`. The name `tf.constant` comes from the symbolic
APIs (like `tf.data` or keras functional models) where the `value` is embeded
in a `Const` node in the `tf.Graph`. `tf.constant` is useful for asserting
that the value can be embedded that way.

**Args**

- value: 由任意数据类型构成的标量或张量

- dtype: `None`时，输出数据类型由`value`数据类型推断得出；否则将`value`转换为指明的数据类型

- shape: `value`为一标量时，则输出为指明`shape`的元素由`value`构成的张量，`value`为张量时，输出`shape`应为*可由原张量 reshape 得到的形状*

- name: 张量名称



Raises:
  TypeError: if shape is incorrectly specified or unsupported.
  ValueError: if called on a symbolic tensor.



**Type**: function

In [66]:
x = tf.constant(True, shape=[2, 3])
x = tf.constant([True, False], dtype = tf.float32)
x = tf.constant([1., 2., 3., 4., 5., 6.], shape=[2, 3])

由于`tf.constant`将值嵌入到了`tf.Graph`，对于符号型张量会抛出异常

>>> i = tf.keras.layers.Input(shape=[None, None])
>>> t = tf.constant(i)
Traceback (most recent call last):
...
NotImplementedError: ...

`tf.constant` will _always_ create CPU (host) tensors. In order to create
tensors on other devices, use `tf.identity`. (If the `value` is an eager
Tensor, however, the tensor will be returned unmodified as mentioned above.)

Related Ops:

* `tf.convert_to_tensor` is similar but:
  * It has no `shape` argument.
  * Symbolic tensors are allowed to pass through.

    >>> i = tf.keras.layers.Input(shape=[None, None])
    >>> t = tf.convert_to_tensor(i)

* `tf.fill`: differs in a few ways:
  *   `tf.constant` supports arbitrary constants, not just uniform scalar
      Tensors like `tf.fill`.
  *   `tf.fill` creates an Op in the graph that is expanded at runtime, so it
      can efficiently represent large tensors.
  *   Since `tf.fill` does not embed the value, it can produce dynamically
      sized outputs.

#  

# tf.ones()

`tf.ones(shape, dtype=tf.float32, name=None)`

__Args__

- shape: 当为一标量时，返回一维张量

- dtype: 略

- name: operation的

__Type__: function

In [4]:
x = 3 * tf.ones(1, dtype=tf.int8, name="test")
x = 5 + tf.zeros([3, 4], dtype=tf.float16, name=None)
print(x)

tf.Tensor(
[[5. 5. 5. 5.]
 [5. 5. 5. 5.]
 [5. 5. 5. 5.]], shape=(3, 4), dtype=float16)


#  

# tf.fill()

`tf.fill(dims, value, name=None)`

__Docstring__

与将取值以`Const`节点嵌入计算图的`tf.constant(value, shape=dims)`不同，`tf.fill`在图运行环境中进行衡量，并支持基于其他运行环境的`tf.Tensors`的动态形状(`tf.fill` evaluates at graph runtime and supports dynamic shapes based on
other runtime `tf.Tensors`)

__Args__

- dims: 取标量时仍返回张量

- value: 略

- name: `tf.Tensor`的


__Type__: function

In [ ]:
x = 4 * tf.fill(1, value=1.2, name=None)  # ≈ np.fill()

#  

#  

<div align=center>

# 张量运算
</div>

# tf.add()

`tf.add(x, y, name=None)`

__Docstring__

`math.add`支持broadcasting，而`AddN`不支持；更多有关 broadcasting
[here](http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)

`tf.subtract()`, `tf.divide()`类似

__Args__

- x: `bfloat16`, `half`, `float32`, `float64`, `uint8`, `int8`, `int16`, `int32`, `int64`, `complex64`, `complex128`, `string`类型的`Tensor`

- y: 与`x`形状相同的`Tensor`.

- name: operation's

__Type__: function

In [ ]:
tf.multiply()
tf.square()
tf.pow()
tf.sqrt()
tf.matmul()

#  

# tf.matmul()
```python
tf.matmul(
    a,
    b,
    transpose_a=False,
    transpose_b=False,
    adjoint_a=False,
    adjoint_b=False,
    a_is_sparse=False,
    b_is_sparse=False,
    name=None,
)
```
**Docstring**:

Multiplies matrix `a` by matrix `b`, producing `a` * `b`.

The inputs must, following any transpositions, be tensors of rank >= 2 where the inner 2 dimensions specify valid matrix multiplication dimensions, and any further outer dimensions specify matching batch size.

If one or both of the matrices contain a lot of zeros, a more efficient
multiplication algorithm can be used by setting the corresponding
`a_is_sparse` or `b_is_sparse` flag to `True`. These are `False` by default.
This optimization is only available for plain matrices (rank-2 tensors) with
datatypes `bfloat16` or `float32`.

由于 python 3.5 及以上的版本支持`@`操作符([PEP 465](https://www.python.org/dev/peps/pep-0465/))，而在 TensorFlow 中其直接等价于`tf.matmul()`函数，故以下两行代码是等价的

```python
    d = a @ b @ [[10], [11]]
    d = tf.matmul(tf.matmul(a, b), [[10], [11]])
```
**Args**:

- a, b: 应为秩大于 1 的`float16`, `float32`, `float64`, `int32`, `complex64`, `complex128`类型的`tf.Tensor`，其中最后 2 维是矩阵乘法发生的维数

- transpose_a, transpose_b: `True`时在计算前先对张量进行转置操作

- adjoint_a: `True`时在计算前先对张量进行共轭转置操作，若`transpose_a`和`adjoint_a`，或`transpose_b`和`adjoint_b`同时设为`True`时会抛出异常

- a_is_sparse, b_is_sparse: `True`时认为张量为一稀疏张量，但不支持`tf.sparse.SparseTensor`类型，即只认为该张量大多元素为 0；更多关于`tf.SparseTensor`的乘法参见`tf.sparse.sparse_dense_matmul`

- name: operation's

**File**:   \tensorflow\lib\site-packages\tensorflow\python\ops\math_ops.py

**Type**:      function

In [ ]:
x = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])
y = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2])
z = tf.matmul(a, b)


A batch matrix multiplication with batch shape [2]:

>>> a = tf.constant(np.arange(1, 13, dtype=np.int32), shape=[2, 2, 3])
>>> a  # 3-D tensor
<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],
       [[ 7,  8,  9],
        [10, 11, 12]]], dtype=int32)>
>>> b = tf.constant(np.arange(13, 25, dtype=np.int32), shape=[2, 3, 2])
>>> b  # 3-D tensor
<tf.Tensor: shape=(2, 3, 2), dtype=int32, numpy=
array([[[13, 14],
        [15, 16],
        [17, 18]],
       [[19, 20],
        [21, 22],
        [23, 24]]], dtype=int32)>
>>> c = tf.matmul(a, b)
>>> c  # `a` * `b`
<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[ 94, 100],
        [229, 244]],
       [[508, 532],
        [697, 730]]], dtype=int32)>



#  

# tf.cast()

`tf.cast(x, dtype, name=None)`

__Docstring__

对于`Tensor`，将`x`数据类型转换为`dtype`，对于`SparseTensor`或`IndexedSlices`，将`x.values`数据类型转换为`dtype`。

`x`与`dtype`支持的数据类型有`uint8`、`uint16`、`uint32`、`uint64`、`int8`、`int16`、`int32`、`int64`、`float16`、`float32`、`float64`、`complex64`、`complex128`、`bfloat16`。当将虚数转为实数时，只返回实部；当将实数转换为虚数时，虚部为0

``name``为运算名称

__Type__

function

In [67]:
x = tf.fill([3, 4], 2.99)
y = tf.cast(x, tf.int8)
print(y)

tf.Tensor(
[[2 2 2 2]
 [2 2 2 2]
 [2 2 2 2]], shape=(3, 4), dtype=int8)


#  

# tf.argmax()

`tf.argmax(input, axis=None, output_type=tf.int64, name=None)`

**Docstring**

`input`只能为`float32`, `float64`, `int32`, `uint8`, `int16`, `int8`, `complex64`, `int64`, `qint8`, `quint8`, `qint32`, `bfloat16`, `uint16`, `complex128`, `half`, `uint32`, `uint64`的`Tensor`；`axis`和`output_type`只能为`tf.int32`、`tf.int64`

Note that in case of ties the identity of the return value is not guaranteed

**similarities**
- tf.argmin()
- 

**File**:  \tensorflow\lib\site-packages\tensorflow\python\ops\math_ops.py

**Type**: function

#  

# tf.reduce_min()

`tf.reduce_min(input_tensor, axis=None, keepdims=False, name=None)`

__Docstring__

等价于`np.min()`

Reduces `input_tensor` along the dimensions given in `axis`.
Unless `keepdims` is true, the rank of the tensor is reduced by 1 for each
entry in `axis`. If `keepdims` is true, the reduced dimensions
are retained with length 1.

If `axis` is None, all dimensions are reduced, and a
tensor with a single element is returned.

__Args__

- input_tensor: 应为实数类型张量

- axis, keepdims: 略

- name: operation的

__Type__

function

In [ ]:
x = tf.constant([[1, 3, 4, 0, 3], [2, 5, 1, 8, 2]])
y = tf.reduce_min(x, axis=0)  # == np.min()
y = tf.reduce_max(x, axis=1)  # == np.max()
y = tf.reduce_sum(x, axis=0)  # == np.sum()
y = tf.reduce_mean(x, axis=0)  # == np.mean()
print(y)
y = tf.reduce_prod(x, axis=0)  # == np.prod()

#  

# tf.reduce_any

`tf.reduce_any(input_tensor, axis=None, keepdims=False, name=None)`

__Docstring__

沿张量某一维度执行`or`计算



For example:

```python
x = tf.constant([[True,  True], [False, False]])
tf.reduce_any(x)  # True
tf.reduce_any(x, 0)  # [True, True]
tf.reduce_any(x, 1)  # [True, False]
```

__Args__

- input_tensor: 布尔型张量

- axis, keepdims, name: 略

__Type__

function

In [ ]:
x = tf.constant(
    [[ True, False, True, False, False, True],
     [False, True, True, False, False, True]])
y = tf.reduce_any(x, axis=0)  # == np.any()
print(y)
y = tf.reduce_all(x, axis=0)  # == np.all()
print(y)

#  

# tf.reduce_logsumexp()

`tf.reduce_logsumexp(input_tensor, axis=None, keepdims=False, name=None)`

__Docstring__

计算

$$\ln \left( \sum\exp(input[shape[i]])\right)$$

该函数比直接计算$\ln\left(\sum\exp(input)\right)$要稳定，其避免了指数函数和对数函数的溢出问题. 

__Args__: 略

__Type__: function

In [ ]:
x = tf.constant([[0., 0., 0.], [0., 0., 0.]])
y = tf.reduce_logsumexp(x)  # log(6)
y = tf.reduce_logsumexp(x, 0)  # [log(2), log(2), log(2)]
y = tf.reduce_logsumexp(x, 1)  # [log(3), log(3)]
y = tf.reduce_logsumexp(x, 1, keepdims=True)  # [[log(3)], [log(3)]]
y = tf.reduce_logsumexp(x, [0, 1])  # log(6)

#  

#  

#  

# tf.GradientTape()
`tf.GradientTape(persistent=False, watch_accessed_variables=True)`

**Docstring**:

创建一个用于记录对变量的操作以便于进行自动微分的上下文管理器，若某操作该上下文管理器中执行，则这些操作将会被记录下来；

**Args**:

- persistent: 用于指明是否创建一个 persistent 的 gradient tape；默认 False，即最多可以调用这个对象上的`gradient()`方法一次

- watch_accessed_variables: 用于指明是否自动追踪上下文管理器中的所有可训练变量，默认为 True；若为 False，则使用者需对每个希望被追踪的变量调用`watch`方法

**File**:    \tensorflow\python\eager\backprop.py

**Type**:           type

**Subclasses**:     LossScaleGradientTape

### Example
通过嵌套计算高阶导数：

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as Gradtape:
    Gradtape.watch(x)
    with tf.GradientTape() as gradtape:
        gradtape.watch(x)
        y = x * x
        dy_dx = gradtape.gradient(y, x)   # => 6.0
    d2y_dx2 = Gradtape.gradient(dy_dx, x) # => 2.0

By default, the resources held by a GradientTape are released as soon as GradientTape.gradient() method is called. To compute multiple gradients over the same computation, create a persistent gradient tape. This allows multiple calls to the gradient() method as resources are released when the tape object is garbage collected. For example:

In [22]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x * x
    z = y * y
dz_dx = g.gradient(z, x)  # 108.0
dy_dx = g.gradient(y, x)  # 6.0
del g

x = tf.Variable(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
    z = y * y
grads = g.gradient([z, y], x)
print(grads)

tf.Tensor(114.0, shape=(), dtype=float32)


By default GradientTape will automatically watch any trainable variables that
are accessed inside the context. If you want fine grained control over which
variables are watched you can disable automatic tracking by passing
`watch_accessed_variables=False` to the tape constructor:

In [ ]:
```python
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(variable_a)
    y = variable_a ** 2  # Gradients will be available for `variable_a`.
    z = variable_b ** 3  # No gradients will be available since `variable_b` is
                       # not being watched.
```

## tf.GradientTape.gradient()
```python
<GradientTape>.gradient(
    target,
    sources,
    output_gradients=None,
    unconnected_gradients=<UnconnectedGradients.NONE: 'none'>,
)
```

**Docstring**:

利用`GradientTape`上下文管理器中记录的操作计算梯度，返回由`Tensor`或`IndexedSlices`或 `None`构成的列表或嵌套结构，每个与`sources`中的元素对应，即返回对象的结构与`sources`相同；若该方法在`GradientTape`上下文管理器中调或在一个非 persistent 的上下文管理器中多次调用则会报错；ValueError: if the target is a variable or if unconnected gradients is called with an unknown value

**Args**:

- target: 要微分的由`Tensor`或`Variable`构成的列表或嵌套结构

- sources: 由`Tensor`或`Variable`构成的列表或嵌套结构，`target`会相对`sources`中的元素进行微分

- output_gradients: 包含了梯度的列表，每一个元素对应`target`中的一个元素，默认 None

- unconnected_gradients: a value which can either hold 'none' or 'zero' and alters the value which will be returned if the target and sources are unconnected. The possible values and effects are detailed in 'UnconnectedGradients' and it defaults to 'none'.

**File**:    \tensorflow\python\eager\backprop.py

**Type**:      function

In [ ]:
tf.GradientTape.gradient()

#  

#  

# tf.convert_to_tensor()

`tf.convert_to_tensor(value, dtype=None, dtype_hint=None, name=None)`

__Docstring__

Converts the given `value` to a `Tensor`.

该函数将各种类型的 Python 对象转化为`Tensor`类型的对象，其支持`Tensor`对象、Numpy数组、Python 列表、Python 标量。当`float`和`string`类型的 Python 列表或标量中出现`None`时，与 Numpy 将`None`转换为数值的行为不同，该函数会抛出异常

**Args**

- value: 满足其类型有已注册的`Tensor`的转换函数的对象(An object whose type has a registered `Tensor` conversion function)

- dtype: 略

- dtype_hint: 用于`dtype`为`None`时；在某些情况下，调用者在转换为张量时可能没有想到dtype，所以dtype_hint可以用作一个软首选项；若目标转换类型无法转换，则该参数等同于无效

- name: 张量名称

**Type**

function

In [ ]:
x = np.arange(12)
y = tf.convert_to_tensor(x)
print(x)
print(y)

#  

#  

# tf.one_hot()

```python
tf.one_hot(
    indices,
    depth,
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None,
)
```
__Docstring__

Returns a one-hot tensor.

`indices`中由索引表示的位置取值为`on_value`，而其他所有位置取值为`off_value`，若期望输出为非数字类型变量，则`on_value`和`off_value`必须指明。

- `indices`为标量时，输出为长度为`depth`的向量；

- 若`indices`为长度为`features`的向量，则输出形状

    - 在axis = -1时为 $features \times depth$

    - 在axis = 0时为 $depth \times features$

- 若`indices`是形状为`[batch, features]`的矩阵，则输出形状
    
    - 在`axis == -1`时为 $batch \times features \times depth$

    - 在`axis == 1`时为 $batch \times depth \times features$

    - 在`axis == 0`时为 $batch \times batch \times features$

- 若`indices`为不规则张量，`axis`必须为正且需指代一个规则的维度，结果等价于将`one_hot`应用在不规则张量的取值上，并生成一个新的不规则张量

__Args__

- indices: 内容代表索引的`Tensor`；若`indices`的秩为 N，则输出数组的秩为 N+1，新的维度会添加在`axis`指明位置中；

- depth: 定义 one hot 维度的标量

- on_value: 输出结果中，当`indices[j] = i`时的取值，默认为 1，需与`off_value`和`dtype`数据类型匹配

- off_value: 输出结果中，当`indices[j] != i`时的取值，默认为 0，需与`on_value`和`dtype`数据类型匹配

- axis: 在shape中添加索引的位置，默认 -1

- dtype: 默认为`tf.float32`，需与`on_value`和`off_value`匹配

- name: operation's

__Type__: function

In [ ]:
x = tf.one_hot(indices=3, depth=7, on_value=6, off_value=-2, dtype=tf.int8)
print(x)

In [ ]:
'''
    axis=0时返回(6,4)数组，对于indices的元素“2”来说，其之前的索引为(1)
    则将索引“2”添加到新索引(x,y)的索引为0的位置，即x的位置，进而得到新索引(2,1)
'''
classes = 6
targets = [1, 2, 4, 3]
x = tf.one_hot(indices=targets, depth=classes, axis=0)
print(x)
'''
    axis=0时返回(6,4)数组，对于indices的元素“2”来说，其之前的索引为(1)
    则将索引“2”添加到新索引(x,y)的索引为1的位置，即y的位置，进而得到新索引(1,2)
''' 
y = tf.one_hot(indices=[5, 2, 4, 3], depth=6, axis=1)

In [ ]:
'''
    axis=0时返回(5,2,3)数组，对于indices的元素“4”来说，其之前的索引为(0,2)
    则将索引“4”添加到新索引(x,y,z)的索引为0的位置，即x的位置，进而得到新索引(4,0,2)
''' 
x = tf.one_hot(indices=[[0, 2, 4], [1, 0, 3]], depth=5, axis=0)
y = tf.one_hot(indices=[[0, 2, 4], [1, 0, 3]], depth=5, axis=1)
z = tf.one_hot(indices=[[0, 2, 4], [1, 0, 3]], depth=5, axis=2)
print(x, y, z, sep="\n\n")

In [100]:
indices = tf.ragged.constant([[0, 1], [2]])  # (2, 1)
x = tf.one_hot(indices=indices, depth=3)  # output: (2, None, 3)
print(x)

<tf.RaggedTensor [[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]], [[0.0, 0.0, 1.0]]]>


#  

#  

In [ ]:
tf.where()